# LX 496/796  Formalizing meaning

In this homework, you will become familiar with creating context free grammars in NLTK and ways to formalize meaning.  Also, parsing, drawing, and traversing syntactic trees.  And we will get started with a little bit of the work on SHRDLU.

## Longer term goal: A tiny version of SHRDLU

A quick word about SHRDLU.  [SHRDLU](https://en.wikipedia.org/wiki/SHRDLU) is Terry Winograd's program from 1971 that could converse about things in its very limited universe of a block world.  You can find a video of it doing its thing [on YouTube here](https://www.youtube.com/watch?v=bo4RvYJYOzI).  You can see it doing a lot of pretty interesting things.  Learning names for things, remembering what it was talking about, moving blocks around, making plans to accomplish goals.  Ours will not be that sophisticated.

In service of this goal, we will need to accept input, parse the trees, and assign a semantics to the parsed sentences so that we can query or act on the blocks world. So there are a couple of pre-requisites.  In this first assignment (of two) we will work on doing the semantic parsing and world representation, and some of the display planning.  The second assignment will get deeper into making it actual work.

# Formalizing meaning #

This largely follows the discussion from chapter 10 in the NLTK book, and stuff we talked about in class, but maybe more elaborated here.  And more doing and less watching.  If it all went smoothly for you in class, it might go pretty smoothly here too.

What we're doing here is setting up a representation of meaning, based on a representation of the world.  We will consider a sentence/statement to be something that can be evaluated as either true or false based on the state of the world.  In order to do these evaluations, we need to have a way to represent the state of the world.

In the world we will work with here first, there are four people: Andrea, Bobby, Chris, and Dana.  These are our *individuals*.
An "individual" need not be a person, it's just some kind of entity that we can refer to.
So, let's also add a couple of non-human individuals as well.  To keep things somewhat simple, they
will be the Moon and the Sun.  We are going to pretend their names are `the_sun` and `the_moon`
however (having a space in there makes things not work, so we will use `_` instead of a space).

So, step one, let's define a set of our individuals.  (There is no intrinsic order to these
individuals, they are just the individuals in our model of the world, so it should be a set and
not a list.) This set is traditionally called the **domain of individuals**, hence the name `dom` for our set.

In [ ]:
# define our individuals, these are people/objects that can be referred to or have properties
dom = {'a', 'b', 'c', 'd', 'm', 's'}

Now we will build up some information about how English words are mapped to these individuals.
First off, we will set up the names.
What we do is
set up a multi-line string first, using then `"""` delimiters on each end.

In [ ]:
# define the mapping from english words => individuals
names = """
andrea => a
bobby => b
chris => c
dana => d
the_sun => s
the_moon => m
"""

As we did in class, and as it is done in the textbook, we
will use NLTK's `fromstring` function to create a `Valuation` from these.

> The `Valuation` is the name of the mapping from English to representations in the model of the world.

In [ ]:
import nltk
# create the Valuation from our names above
val = nltk.Valuation.fromstring(names)
# make sure that it worked by printing the Valuation back out again
print(val)

Now that we have done this, we can "evaluate" the English words and get their referents.

In [ ]:
print(val['bobby'])
print(val['the_moon'])

This tells us which individual in our set of individuals is being referred to by "bobby" and by "the moon".
Doing this counts, at least in a certain sense, as translating from English into "semantics."  We are
determining the meaning of the word "bobby," for example.

The individuals in this world have properties and relationships, however, as well.  For example, some of the
individuals are people.  So, we define "person" as being something that holds of the individuals
a, b, c, and d.  For the moment, we are going to create a new temporary Valuation to hold this information,
and we will merge this in with our primary valuation function `val` shortly.

In [ ]:
valp = nltk.Valuation.fromstring("person => {a, b, c, d}")
print(valp)

It's kind of a pain to keep typing this `nltk.Valuation.fromstring` thing, let's give it a shorter
name.  I'm going with `vfs` for "valuation-from-string".  After doing this, we can just type `vfs` everywhere we would have typed `nltk.Valuation.fromstring` otherwise, it's basically just a nickname/alias.

In [ ]:
vfs = nltk.Valuation.fromstring

Now, the sun and the moon are not people, what are they?  Let's make a little temporary valuation for them.

In [ ]:
valsb = vfs("spaceball => {s, m}")

So far, we have three different Valuations (`val`, `valp`, and `valsb`).  Just to be sure, we will print them out.

In [ ]:
print(val)
print(valp)
print(valsb)

But ultimately we want a single Valuation that we can use, so we want to merge
them together into one.  We'll consider `val` to be our main one, and will merge the other two into `val`.  It is possible to combine two Valuations using the `update` function.  So, let's
add `valp` and `valsb` to `val`. First, we'll add the spaceballs one and see what the effect was.

In [ ]:
val.update(valsb)
print(val)

Good, that's what we wanted, now let's add the people.

In [ ]:
val.update(valp)
print(val)

>The `update` function is actually fairly general.  It is defined for Valuations, but it is
>also defined for just regular sets, as well as for dictionaries.  If you call `update` on a
>set or a dictionary, it merges the argument of `update` into it (with priority given to the
>additions, if there is a conflict).

>```python
>aset = {1, 2, 3}
>aset.update({3, 4, 5})
>print(aset) # => {1, 2, 3, 4, 5}
>adict = {'a': 1, 'b': 2}
>adict.update({'b': 4, 'c': 6})
>print(adict) # => {'a': 1, 'b': 4, 'c': 6}
>```

Now, all of our world-building work to date is represented in `val`.  Let's do a little bit more building.
Let us suppose that Andrea and Bobby are from Boston, while Chris and Dana are from Cambridge.  Instead of creating and naming temporary Valuations first and then merging them in, we can skip the naming step and update `val` directly with the output of `vfs`.

In [ ]:
val.update(vfs("bostonian => {a, b}"))
val.update(vfs("cantabrigian => {c, d}"))

Now, we've defined the mapping between names and individuals, and we've defined some nouns/predicates
that hold of sets of individuals.  What remains is to define some relationships between them.
Relationships are asymmetrical, so just because Andrea likes Bobby does not mean that Bobby likes Andrea.
But let's start with that.

In [ ]:
val.update(vfs("likes => {(a, b)}"))

Ok, now let's (attempt to) make it mutual.  This isn't going to work, but it seems like it might have.  We're trying to add "b likes a" (Bobby likes Andrea) to `val`, and we know that `update` can add things to `val`.  But when we try it, we don't get the mutual liking we had hoped to get.

In [ ]:
val.update(vfs("likes => {(b, a)}"))
print(val['likes'])

Specifically, the result was that---even though Bobby now likes Andrea---Andrea no longer likes Bobby.

What happened of course is that we did not *add* a pair to the liking relation, but rather just replaced it with a different pair.  When we `update` a valuation function, it will replace the entire value for the words we include in our update argument.

So to get the mutual liking we wanted, we could simply spell it out, update `val` with definition of "likes" that contains both pairs:

In [ ]:
val.update(vfs("likes => {(b, a), (a, b)}"))
print(val['likes'])

And that gets what we want.  But it would be nice to be able to add relations in stages, rather
than redefine the relation in full every time.  And we can, we just have to be wiser about what we update.

What we were trying to do, really, was update the value for "likes" (which in turn is an update to `val` as a whole).  It turns out that things are set up such that if you update an individual *value*, it will behave like updating a regular set.  So we *can* accomplish the thing we were trying to do (just add a single pair without removing all other pairs as well), we just need to update `val['likes']` instead of updating `val`.

**BUT NOTE**: there is a difference between the notation in the string that `vfs` parses into a Valuation data structure and the notation that Python uses in the eventual structure.  Above, in the arguments to `vfs`, we were able to refer to "a likes b" as `{(a,b)}`.  But when we look at the values of `val['likes']` we see that they are in the form `{('a', 'b')}`.  Specifically, in the string we give to `vfs`, it will just assume the members of the pairs are strings.  But if we are updating the `val` structure without the help of `vfs`, then we need to explicitly mark the individuals as being strings by putting quote marks around them.

Given all that, it's pretty easy to do these additive updates now.

In [ ]:
# reset it to just Andrea likes Bobby and see if we can add the reverse
val.update(vfs("likes => {(a, b)}"))
print(val['likes'])
# now try to add the reverse
val['likes'].update({('b', 'a')})
print(val['likes'])

Ok, that appears to have worked.  So, if you want to add a particular relationship or individual to a predicate, you can address the predicate and `update` it.

Now, let's finish setting up the world (with respect to likings).  Here is the goal state:

- Andrea likes everyone
- Everyone likes Dana
- Bobby likes Andrea
- Dana likes Chris
- Andrea and Bobby like the Sun
- Chris and Dana like the Moon

To get you started, we'll do "Andrea likes everyone" because it also allows me to remind you that for something like this we can actually do this somewhat programmatically.  We know who all the people are, they're the people referred to in `val['person']`.

In [ ]:
# go through everyone "person" refers to, add them to Andrea's "like" list:
for (x,) in val['person']:
    val['likes'].update({('a', x)})
# display the result to see if it worked
print(val['likes'])

## TASK 1 Finish setting up the world

Finish setting up `val['likes']` to represent the world situation described above.


In [ ]:
# Answer 1: Finish setting up val['likes']


If it worked, the command below will succeed (that is, it will print no error).  If it stops with an error, then you haven't quite gotten the definition of `val['likes']` right yet.

> If you are comfortable enough with Python to see how this verification below works, then you will also see how you might be able to use the same trick to set up the world in a compact way as well, by using `set`, `zip`, and strings, and setting `val['likes']` to the result.  Doing it this way is harder to read/grasp when you go back to the code later, of course.  It basically goes through the strings and makes pairs drawing the first element from the first one and the second element from the second one.  So the first pair it makes is `('a', 'a')` and the last one is `('d', 'm')`.  

In [ ]:
assert val['likes'] == set(zip("aaaaabcdbdabcd", "abcdddddacssmm")), \
    "World is not set up properly yet."

At this point, your valuation function (`val`) will have passed the test for being set up, but let's do a more transparent test on this world.  We know it is supposed to be the case that Andrea likes everyone (that is, all the people).  How can we verify that part?  (Short of just looking at `print(val['likes'])` and checking by hand whether all the expected pairs are there.)

If you type the following, you should get `True` if Andrea likes all the people.

In [ ]:
not False in [(val['andrea'], x) in val['likes'] for (x,) in val['person']]

Got `True`? Great. But why? If you just blindly executed the cell and got `True` without figuring
out what it is doing, that's fine.  But now we're going to figure out what it is doing.

First of all, remind yourself what `val['andrea']`, `val['person']`, and `val['likes']` are:

In [ ]:
print(val['andrea'])
print(val['person'])
print(val['likes'])

When we say `for (x,) in val['person']` we are extracting each 1-tuple from the `val['person']` list, and then calling the member of that tuple `x`.  If we made a list of those, it would be a list of the individuals that "person" is true of.

In [ ]:
[x for (x,) in val['person']]

We are trying to determine whether Andrea likes all the people.  So, we check, for each person,
whether it is true that Andrea likes that person.  When we're done checking people, we should not
have found any that yield `False`.  As you just saw, `val['person']` is a set of 1-tuples, like `('a',)`.
So to go through the people, we want to use `for (x,) in val['person']` in order to set `x` to be
the individual in our domain that corresponds to the person (e.g., `'a'`).  To determine whether
Andrea likes the person in `x`, we need to find out whether the pair that has Andrea as the first
member and `x` as the second member is in the set of "likings" in `val['likes']`.  The individual
that Andrea represents is `val['andrea']` (which will be `'a'`).  So, we evaluate whether the
pair `(val['andrea'], x)` is in `val['likes']`.  The expression `(val['andrea'], x) in val['likes']`
will be `True` if Andrea likes `x` and `False` otherwise.  The list that this list comprehension
builds will be a list of `True` or `False` values (one for each person).  If Andrea indeed likes every person, then the
list should be `[True, True, True, True, True]`.  Finally, we check to see if `False` is anywhere in that
list.  If it is, then: Andrea doesn't like every person.  If there is no `False` in there,
then Andrea *does* like every person.  So, `not False in [...]` is `True` achieves the desired evaluation.

## TASK 2 Everyone likes Dana, in Python

Use the same technique to verify that every person likes Dana.

In [ ]:
# Answer 2: Verify that every person likes Dana


Now, let's formalize our model of the world into an official NLTK model.  A model pairs a domain and a Valuation function.

In [ ]:
m = nltk.Model(dom, val)

Once we have a model defined, we can use the model's `evaluate` function to test the truth
of things in the model.  In order to use `evaluate` we also need to set up an "assignment function"
(which can be thought of as a record of who we're pointing to).  To begin with, we'll just set up
an empty assignment function (we aren't pointing at anything).  So we give it `dom` (the things we could be pointing to in principle), but don't actually point at anything.

In [ ]:
g = nltk.Assignment(dom)

Now, we can verify that Dana likes Chris, and verify that Bobby does not like Chris, in the following way:


In [ ]:
print(m.evaluate('likes(dana, chris)', g))
print(m.evaluate('-likes(bobby, chris)', g))

This is the same as just checking if `('d', 'c') in val['likes']` and `('b', 'c') not in val['likes']` but using `evaluate` allows us to write things in more concise logical form.

## TASK 3 Evaluate and likings

Use `evaluate` to verify that Dana does not like Bobby, and that Chris likes the Moon.

In [ ]:
# Answer 3: Verify that Dana does not like Bobby, that Chris likes the Moon


We can also use quantifiers like `all` and `exists` with `evaluate`.  For example, we can re-verify
that Andrea likes every person, like so:

In [ ]:
print(m.evaluate('all x.(person(x) -> likes(andrea, x))', g))

The way the evaluation above works is pretty much exactly how our home-spun version from Task 2 worked.  It goes through
all of the individuals in the domain one by one, and for each it checks to see if it's a person,
and if it is a person, then it checks to see if it is the second member of a pair, whose first member
is Andrea, that can be found in the list of "likings".

## TASK 4 Evaluate: everyone likes Dana

Use `evaluate` to verify that everybody likes Dana.

In [ ]:
# Answer 4: Verify that everybody likes Dana (using "all x")


You can also use `exists`, which is true if the condition is met for at least one of the individuals in the
domain.  So, if we want to ascertain that at least *somebody* likes Bobby, we can do the following:

In [ ]:
print(m.evaluate('exists x.(person(x) & likes(x, bobby))', g))

What that means is that we can find *some* `x` in our domain `dom` such that `x` is both a `person` and
in a `likes` relation with Bobby.

## TASK 5 Evaluate: liking the Sun, being from Cambridge

Use `evaluate` to verify that every Bostonian likes the Sun, and that no spaceballs are from Cambridge.

In [ ]:
# Answer 5: Verify that every Bostonian likes the Sun, no spaceballs are from Cambridge


The string that we give to `evaluate` is first interpreted as a "semantic Expression"
built from a string.  We can define such expressions
directly for other uses, apart from just evaluating their truth.  The function that does this is `nltk.sem.Expression.fromstring`.  Like before, we'll
give it a shorter name (`sfs`) to save on some typing.  Then we'll define a formula `f1` to be
"x likes the Moon".

In [ ]:
sfs = nltk.sem.Expression.fromstring
f1 = sfs('likes(x, the_moon)')
print(f1)

So, is "x likes the Moon" true?  No idea.  We can't decide that until we know who `x` is
supposed to be.  Once we know who `x` is, then we can figure out whether it's true.  Because
we don't know who `x` is, `x` is considered a "free variable."  Although it's kind of obvious,
we can interrogate `f1` to ask it what its free variables are:

In [ ]:
print(f1.free())

If we want to know who likes the Moon, we can ask the model to tell us which individuals,
when substituted in for `x`, would make `f1` true:

In [ ]:
print(m.satisfiers(f1, 'x', g))

## TASK 6 Satisfiers liked by Chris

Use `satsifiers` to determine who/what Chris likes.

In [ ]:
# Answer 6: Use satisfiers to determine who/what Chris likes


One way that we can set a value for `x` is to use `x` to point to an individual.  That is,
suppose we point (with our "x" finger) at Bobby, and then ask whether "x likes the Moon"
is true.  Since this tells us who `x` is (namely, Bobby), we can decide whether "x likes the Moon"
is true.  It's true if (and only if) Bobby likes the Moon.

In [ ]:
f1 = sfs('likes(x, the_moon)')
# reminder that f1 is the expression x likes the moon
print(f1)
# point to Bobby with x
g['x'] = 'b'
# does x like the moon, given that x is Bobby?
print(m.satisfy(f1, g))
# how about if x is Chris?
g['x'] = 'c'
print(m.satisfy(f1, g))

This is what the assignment function is for.  It is a record of who/what we are pointing at,
and with which fingers.  This is really designed to handle pronouns like *he*, *she*, *it*.
If you use those pronouns, it is assumed that something in the discourse is basically pointing
at the individual you mean.  Without some kind of pointing ("deixis") you won't be able to
interpret the referent of a pronoun.



# Parsing sentences

Let's try to build a little grammar that can take sentences and interpret them.  What we
want to do here is create some phrase structure rules that will apply the semantics we
defined to a syntactic structure.  We'll build this up from the bottom.


In [ ]:
# first we install svgling so we can draw illustrative trees
!pip install svgling
import svgling

To get a visual aid of where we're going, we'll use `svgling` to draw trees.  So that's why we just installed it.  

First, a quick note on the `svgling`-friendly tree format.  It is very simple.  A node is a tuple, where the first element is the label of the subtree, and the subsequent elements are the daughters.  A daughter can be a string or a label.  A multi-line label is possible if you put `"\n"` in the string (newline).

Here is a tree showing the simple version of what we're heading for, and filling in the semantics for the DPs, which we are about to define.


In [ ]:
# The tree, with the semantics of DP marked
extree1 = ("S", ("DP\nchris","chris") , ("VP", ("V", "likes"), ("DP\ndana", "Dana")))
svgling.draw_tree(extree1)


As a first step, we will define the DPs, which will be just the names we have.
(We are going to build a big multi-line string and then create the grammar using a
`fromstring` function.)

> Note: From here we'll start calling the subjects, objects, etc. "DPs" (determiner phrases) rather than "NPs" (noun phrases).  It is pretty well established in syntax that the determiner is the proper head of these phrases.  This requires assuming that there are silent determiners in some cases, though for now we will be content with just considering names to be DPs without any further internal structure.  When we start working with quantifiers like "every person" the interal structure will become more relevant.

In [ ]:
dpdef = r"""
DP[SEM=<andrea>] -> 'andrea'
DP[SEM=<bobby>] -> 'bobby'
DP[SEM=<chris>] -> 'chris'
DP[SEM=<dana>] -> 'dana'
DP[SEM=<the_sun>] -> 'the_sun'
DP[SEM=<the_moon>] -> 'the_moon'
"""

What this means is that if the English word 'andrea' is encountered, that can be
interpreted as an NP with the SEM feature being `<andrea>`.  And likewise for the other
proper names.

> **NOTE** Before the `"""` there is an `r`.  This is actually somewhat important later, because it means that the `\` (backslash) character will be interpreted as a backslash.  If we didn't have then `r` ("raw"), then Python would do some magic that would wind up treating the `\` character as an "escape" character.

As for how the whole tree combines, it will start with `S` at the top, which is
formed from an `DP` and a `VP`, and the `VP` is formed from a `V` and an `DP`.
For now, that's all we'll do.

What we want is for the semantics of the VP to combine the semantics of the V
with the semantics of the DP.  So, if the V is "likes(x, y)", and the DP is "bobby",
then we want the VP to be "likes(x, bobby)", more or less.

Let's start with the VP.  The tree below is what we're looking for.  The semantics for V takes two arguments, the first (outer) one being called `y`, and the second (inner) one being called `x`.  When *likes* combines with *dana*, by applying the function represented by V to the argument represented by the DP, this results in `dana` (the value of the DP) being substituted in for `y`.  The value of the VP thus becomes `\y.likes(y,dana)`. 

In [ ]:
# The tree, with the semantics of V and VP marked
extree2 = ("S", ("DP\nchris","chris") , \
           ("VP\n\\x.likes(x,dana)", ("V\n\\y.\\x.likes(x,y)", "likes"), \
            ("DP\ndana", "Dana")))
svgling.draw_tree(extree2)

The rules we want to add into our grammar are those below, the lines defining VP and V as illustrated in the tree above.

In [ ]:
vpdef = r"""
VP[SEM=<?v(?obj)>] -> V[SEM=?v] DP[SEM=?obj]
V[SEM=<\y.\x.likes(x,y)>] -> 'likes'
"""

Then the last step is to combine the VP and subject DP.  Even in the tree above you can already see how this should work.  The semantics of the VP is a function that takes a single argument.  If we apply the VP function to the subject DP argument, the result is the expression you get by substituting in `chris` for `x`, yielding `likes(dana,chris)`.

We will define `cfgdef` as our overall grammar, and then we will add `vpdef` and `dpdef` (defined above) in as well.

In [ ]:
cfgdef = r"""
% start S
S[SEM=<?vp(?subj)>] -> DP[SEM=?subj] VP[SEM=?vp]
"""

The way to understand this is: The semantics of S is the function that
we get from the semantics of VP, applied to the argument that we get from
the semantics of the DP subject.  So, by saying `DP[SEM=?subj]` we are
naming the value of the DP's `SEM` feature (whatever it is) as `?subj`.
We name the value of the VP's `SEM` feature (whatever it is) as `?vp`.
We assume that `?vp` is a function that can take `?subj` as an argument.
And so, the `SEM` feature that we assign to S is whatever we get when
we apply the function `?vp` to the argument `?subj`.

> Note that the `% start S` is actually important (it is not a comment).  The grammar needs to start with that, this tells the parser what symbol is at the top of the tree.

We then do the same thing for the VP.  We assume that the V is going
to be a function that we can apply to the DP.

> Above was where the `r` was important, because we have some `\` characters in the string.  We need those `\` characters to stay there.  They are being treated as "lambda" characters.  We may not have discussed these yet.  For the moment, maybe just trusting the notation above is wisest.  What the second line says, though, is really: "given a y, and given an x, x likes y".  The `\` indicates that the expression takes an argument, and the fact that there are two means that we need two arguments (liker and likee) before we know whether the liking relationship truly holds.

So, now we can add in the VP and DP definitions we did earlier, and
take a look at the whole grammar.

In [ ]:
cfgdef += dpdef + vpdef
print(cfgdef)

Now that we have the definition, we can parse it into an actual grammar
that NLTK can use, and then connect it to a parser (we will use the one
called `FeatureChartParser`).

In [ ]:
from nltk import grammar
gram = grammar.FeatureGrammar.fromstring(cfgdef)
cp = nltk.FeatureChartParser(gram)

And now we can parse some sentences.  Let's start with "bobby likes chris":

In [ ]:
parses = list(cp.parse('bobby likes chris'.split()))
print(len(parses))
print(parses[0])

If everything worked up to now, you should see that there is 1 parse,
and `print(parses[0])` will show you the parse it got.

The very first line is the overall semantic value for the tree, which we
can get like this:

In [ ]:
treesem = parses[0].label()['SEM']
print(treesem)

And, now that we have this expression, we can test it against the model
to see if it is actually true.  Note that we are using `satisfy` and not
`evaluate` -- the `evaluate` function takes a string and turns it into a
semantic expression, and then calls `satisfy`.  Since we already have a
semantic expression, we can just call `satisfy` directly.

In [ ]:
print(m.satisfy(treesem, g))

And thus we learn that, in this model, Bobby does not like Chris.

If we want to know if Bobby likes Dana, we just change the sentence.

In [ ]:
parses = list(cp.parse('bobby likes dana'.split()))
print(parses[0])
treesem = parses[0].label()['SEM']
print(treesem)
print(m.satisfy(treesem, g))

## TASK 7 Checking the model for truth

Use this grammar to parse sentences telling you whether Chris likes Bobby and
whether Bobby likes the Sun.

> Don't forget that the Sun is all one word (`the_sun`) in this grammar.


In [ ]:
# Answer 7: Parse sentences and check against the model for:
# Chris likes Bobby, and for Bobby likes the Sun



That's actually pretty cool.  We can get from a sentence to a tree to truth conditions to
an actual evaluation of whether a sentence is true or false.  Granted, we can't do very
complicated sentences, but we have a place to start and we can kind of see how we could
proceed.

# Drawing trees

We can use `svgling` to draw trees, but the trees that the `FeatureGrammar` produces are not by default in a format that `svgling` can draw.  So, we will convert them.

Below, a `convert_tree` function is defined that will traverse a parse tree and create a tuple suitable for drawing with `svgling`.  I will explain what the code is doing, but there isn't anything that really relies on your understanding it too deeply.  The idea in "pseudocode" is basically this:

```
convert_tree(node):
  if node is a subtree:
    label = the subtree's label and SEM value
    return (label, convert_tree(x1), ...) where x1, ... are the daughters of node
  else (if node is not a subtree, but rather a string):
    return node
```

This function is recursive.  If one of the daughters of the node/tree we pass in is itself a subtree (a node that has daughters), then `convert_tree` will first be used to convert that subtree, and then included in the return value.  If `convert_tree` is called with just a string (not a subtree), it will just return that string.

Here, then, is the actual code:

In [ ]:
# convert a feature tree into something svgling can draw
def convert_tree(node):
  nodetype = type(node).__name__
  if nodetype == 'Tree':
    keylist = list(node.label().keys())
    label = node.label()[keylist[0]]
    if 'SEM' in node.label():
      label += "\n" + str(node.label()['SEM'])
    return tuple([label] + [convert_tree(child) for child in node])
  else:
    return node

and this is what it does.  We'll parse "Bobby likes Chris", look at the original tree, and the converted tree.

In [ ]:
parses = list(cp.parse('bobby likes chris'.split()))
converted_parse = convert_tree(parses[0])
print("Parse in original form:")
print(parses[0])
print("Parse in converted form:")
print(converted_parse)

And with that, we are in a position to draw the tree.  Which we can do by calling `svgling.draw_tree()` like so:

In [ ]:
# draw the tree
svgling.draw_tree(converted_parse)

I find the tree much more helpful in visualizing how the meaning is being computed, and how the semantic pieces need to be defined to get what we want.  The ultimate goal is to create a formula at the top of the tree that we can evaluate against the model to see if it is true or false.


# Quantifiers, first step

Now, we will add "every person" to our grammar, to see how this works.  We will continue the work on quantifiers more in the next homework.

Earlier, we worked out how to evaluate "everyone likes Dana" both in basic Python and as an expression using `evaluate`.  Now what we want to do is create a grammar that will take a sentence like "every person likes dana" and turn it into the expression we got before, namely `all x.(person(x) -> likes(x, dana))`.  If we can get the SEM feature of the top node of the tree to hold that expression, then we can use `evaluate` to get the truth value.

The first step then is to work out what the structure of the quantifier is.  We know that it is a DP, since it can be a subject.  It is reasonable to take "every" to be the D and "person" to be an NP, in a structure that looks like this:

In [ ]:
svgling.draw_tree(('DP', ('D', 'every'), ('NP', 'person')))

We pretty much have to assume that the meaning of "person" should be a function that is true of people and false of other things.  So, the meaning of "person" is pretty obvious.

In [ ]:
np2def = cfgdef + r"""
NP[SEM=<\y.person(y)>] -> 'person'
"""

And we already basically know what the VP should mean.  True of dana-likers.  We can already compute this with the grammar that we had, so it will be:

`\y.like(y,dana)`

The trick now is how to combine the meanings of the VP and the meaning of the subject to get to the expression we want.

In [ ]:
svgling.draw_tree(('S\nall x.(person(x) -> likes(x, dana))', ('DP', \
  ('D', 'every'), ('NP', 'person\n\\y.person(y)')), \
  ('VP\n\\y.like(y,dana)', ('V', 'likes'), ('DP', 'dana'))))

It seems pretty clear that if we want `all x.(...)` at S, and it's not coming from the VP (which it isn't), then it must be part of the meaning of the DP.

But the `likes(x,dana)` part of S should be coming from VP.  So, what we need to do is assume that DP is a function that can take the VP *as a function* and incorporate it into the more complex expression.

Specifically, we want DP to look like:

```
\Q.all x.(person(x) -> Q(x))
```

The way to understand that is that it takes a function as its argument, calls it `Q`, and then applies that function to `x` inside its expression.

In [ ]:
svgling.draw_tree(('S\nall x.(person(x) -> likes(x, dana))', \
  ('DP\n\\Q.all x.(person(x) -> Q(x))', \
  ('D', 'every'), ('NP', 'person\n\\y.person(y)')), \
  ('VP\n\\y.like(y,dana)', ('V', 'likes'), ('DP', 'dana'))))

And now we need to figure out what the semantics of *every* should be.  The same logic we just followed would suggest that the `\y.person(y)` from the NP is contributing the `person(x)` part of the DP semantics, and the rest of it is coming from D.  So, again we want to make a function that takes a function as its argument, and uses it inside the resulting expression.

The resulting expression at DP is:

```
\Q.all x.(person(x) -> Q(x))
```

So the expression for D would be:

```
\N \Q.all x.(N(x) -> Q(x))
```

So, it takes a predicate (like "person") and calls it `N`, then applies it inside the expression on the left side of the conditional.

> Note: For whatever reason, you don't use the `.` between multiple lambda arguments.  This was true back when we defined "likes" as well.  There's a space here between the `\N` and the `\Q` rather than a `.`.

So now, let's make a new grammar with our old grammar plus the new definitions to allow for "every person" to be a subject.

In [ ]:
cfg2def = cfgdef + r"""
NP[SEM=<\y.person(y)>] -> 'person'
D[SEM=<\N Q.all x.(N(x) -> Q(x))>] -> 'every'
DP[SEM=<?det(?np)>] -> D[SEM=?det] NP[SEM=?np]
S[SEM=<?subj(?vp)>] -> DP[SEM=?subj] VP[SEM=?vp]
"""

In [ ]:
gram2 = grammar.FeatureGrammar.fromstring(cfg2def)
cp2 = nltk.FeatureChartParser(gram2)

If we parse "every person likes chris" we now get two parses.  That's because we have two `S` rules, two ways that we can combine the subject DP and the VP.  One for when the subject is a name, and one for when the subject is a quantifier.

In [ ]:
parses = list(cp2.parse('every person likes dana'.split()))
converted_parses = [convert_tree(p) for p in parses]
print(len(converted_parses))

Let's look at the two parses.

In [ ]:
svgling.draw_tree(converted_parses[0])

In [ ]:
svgling.draw_tree(converted_parses[1])

The first one doesn't really make any sense.  It has combined the subject and VP in the wrong direction.  The second one is the one we want.  We will pursue this more in the next homework.  For now, let's just evaluate whether the second one is true or not. 

> Note here that we are using `satisfy` rather than `evaluate` because the value of the label's SEM feature is already an NLTK Expression.

In [ ]:
treesem = parses[1].label()['SEM']
print(m.satisfy(treesem, g))

It is.


## TASK 8 Every Bostonian likes the_sun

This is just an exercise in tweaking the above discussion slightly, just to help reinforce it.  The goal: add "bostonian" as a noun to the grammar such that you can parse "every bostonian likes the_sun" and evaluate its truth.  It should be true. 


In [ ]:
# Answer 8


# Advanced/Optional - object quantifiers and names

If you got through all of this so quickly it didn't seem interesting, you can try to get a jump on what we'll do next.  What we've got so far is the ability to parse "every bostonian likes the_sun" and "every person likes dana".

But, there are a couple of major problems.  One is that it comes up with a nonsensical parse (alongside the correct parse) now.  It would be better if it came up with just one, that was interpretable.  And: it cannot handle "dana likes every bostonian" (that is: a quantifier in object position).

To handle the first one, think about how you might change the grammar so that the subject DP *always* takes the VP as an argument.  That is, essentially, turn proper names into a quantifier too.  Can you set it up so that there is just one rule that starts `S ->` that combines the subject and VP and can parse both "every person likes dana" and "chris likes dana", giving one parse for each?

And then: what will it take to allow for a quantified object, as in "Dana likes every person"?  We're going to want to do this if we have changed all the names into quantifiers too.  This is potentially very difficult.  The meaning we want for "Dana likes every person" is `all x.(person(x) -> like(d,x))` --- can you get that to come out as a meaning at the top?  What would it require the meaning of "likes" to be?